In [586]:
from importlib import reload

import DataProvider
import Utils

reload(DataProvider)
reload(Utils)

<module 'Utils' from '/home/vincent/vincent/sharedwork/data/market-hedging/Utils.py'>

In [536]:
df = DataProvider.get_random_securities(10, "2013-10-01", "2018-10-01")
df_spy = DataProvider.get_securities("SPY", "2013-10-01", "2018-10-01")

In [537]:
df = Utils.attach_return(df, "symbol")
df_spy = Utils.attach_return(df_spy, "symbol")

In [ ]:
df = df.reset_index().merge(
    df_spy.reset_index().drop("symbol", axis=1),
    on="date",
    suffixes=["", "_spy"]
).dropna().set_index(df.index.names)

In [548]:
df = df.reset_index().merge(
    df_spy.reset_index().drop("symbol", axis=1),
    on="date",
    suffixes=["", "_spy"]
).dropna().set_index(df.index.names)

In [552]:
df.sample(10)

adjClose    return  adjClose_spy  return_spy
symbol date                                                      
PAR    2011-09-26    3.100000  0.006494    100.886032    0.023780
CACC   2017-10-10  282.920000  0.013614    250.064447    0.002638
       2015-08-24  188.390000 -0.023127    177.656926   -0.040884
BOCH   2013-06-20    4.477488  0.000000    142.943678   -0.024778
GRAM   2014-03-21   16.840795 -0.046088    170.343380   -0.003848
PAR    2012-04-16    4.840000  0.018947    120.224418   -0.000656
QUMU   2016-05-25    3.720000  0.019178    199.399291    0.006783
KAR    2017-01-19   43.187399 -0.013086    218.749612   -0.003705
QUMU   2011-02-07   13.006591 -0.001392    112.903123    0.006252
BOCH   2013-02-05    4.505733  0.018367    134.855380    0.010131

In [587]:
Utils.attach_beta(df, "return_spy", "return", "symbol", 10)

,,adjClose,return,adjClose_spy,return_spy,beta
symbol,date,,,,,
QUMU,2010-01-05,15.145655,-0.043503,95.339988,0.002647,0.000000
SOHU,2010-01-05,59.780000,0.009286,95.339988,0.002647,0.000000
PAYX,2010-01-05,22.398291,-0.000649,95.339988,0.002647,0.000000
DPZ,2010-01-05,7.594843,0.009217,95.339988,0.002647,0.000000
KAR,2010-01-05,11.693140,0.002863,95.339988,0.002647,0.000000
BOCH,2010-01-05,4.318711,0.001898,95.339988,0.002647,0.000000
PAR,2010-01-05,5.930000,-0.018212,95.339988,0.002647,0.000000
CACC,2010-01-05,43.800000,-0.000912,95.339988,0.002647,0.000000
QUMU,2010-01-06,14.701527,-0.029324,95.407111,0.000704,-7.297314


In [ ]:
df.sample(10)

In [422]:
# def gm(aDF,Const):
#     v = ((((aDF.A+aDF.B)+1).cumprod())-1)*Const
#     return (aDF.index[0],v.iloc[-1])

# S = pd.Series(dict([ gm(df.iloc[i:min(i+51,len(df)-1)],5) for i in range(len(df)-50) ])); S

# def gm(aDF,Const):
#     v = ((((aDF.A+aDF.B)+1).cumprod())-1)*Const
#     return (aDF.index[0],v.iloc[-1])

def rolling_regression(df, window):
    
    col_X = "return_spy"
    col_Y = "return"
    model_ols = linear_model.LinearRegression()
    
    def reg(d):
        model_ols.fit(
            d[col_X].values.reshape(-1,1),
            d[col_Y].values.reshape(-1,1)
        )
        return (d.index[0], model_ols.coef_[0][0])

    return pd.Series(dict([
                reg(df.iloc[(max(0,i-window+1)):(i+1)]) for i in range(len(df))
            ]))

In [494]:
def rolling_regression(df, col_X, col_Y, group_col=None, window=1):
    
    model_ols = linear_model.LinearRegression()
    
    def reg(d, group=None):
        model_ols.fit(
            d[col_X].values.reshape(-1,1),
            d[col_Y].values.reshape(-1,1)
        )
        if group:
            return ((group, d.index[-1]), model_ols.coef_[0][0])
        else:
            return (d.index[-1], model_ols.coef_[0][0])

    if group_col:
        l = []
        for name, group in df.groupby(group_col):
            curr = pd.Series(dict([
                        reg(group.iloc[(max(0,i-window+1)):(i+1)], name) for i in range(len(group))
                    ]))
            l.append(curr)
        output = pd.concat(l)
        output.index.names = [group_col] + df.index.names
        return output
    else:
        output = pd.Series(dict([
                    reg(df.iloc[(max(0,i-window+1)):(i+1)]) for i in range(len(df))
                ]))
        output.index.names = [group_col] + df.index.names
        return output.set_names(df.index.names)

In [472]:
m.set_index("date").index.names

'date'

In [497]:
o = rolling_regression(m.set_index("date"), "return_spy", "return", "symbol", 5)
o

symbol  date      
AAWW    2010-01-05     0.000000
        2010-01-06    49.875324
        2010-01-07    15.577725
        2010-01-08    14.534465
        2010-01-11    12.586935
        2010-01-12     0.407449
        2010-01-13     1.040716
        2010-01-14     1.037622
        2010-01-15     1.588606
        2010-01-19     1.081938
        2010-01-20     1.456412
        2010-01-21     0.962610
        2010-01-22     0.834953
        2010-01-25     0.811892
        2010-01-26     0.736320
        2010-01-27     1.231931
        2010-01-28     1.414060
        2010-01-29     1.199650
        2010-02-01     1.297712
        2010-02-02     1.288180
        2010-02-03     1.374734
        2010-02-04     0.248908
        2010-02-05     0.200275
        2010-02-08     0.020963
        2010-02-09    -0.371077
        2010-02-10    -0.371353
        2010-02-11    -0.574879
        2010-02-12    -0.516120
        2010-02-16     1.730252
        2010-02-17     2.235393
                     

In [500]:
o.reset_index()[o.reset_index()["date"] == "2010-01-05"]

,symbol,date,0
0,AAWW,2010-01-05,0.0
2972,CRY,2010-01-05,0.0
5397,HA,2010-01-05,0.0
7617,KB,2010-01-05,0.0
9837,MFB,2010-01-05,0.0
10782,NNI,2010-01-05,0.0
13002,SURG,2010-01-05,0.0


In [ ]:
def roll(df, w, group, colx, coly):
    for name, group in df.groupby(group):
        

In [349]:
def roll(df, w):
    roll_array = np.dstack([df.values[i:i+w, :] for i in range(len(df.index) - w + 1)]).T
    panel = pd.Panel(roll_array, 
                     items=df.index[w-1:],
                     major_axis=df.columns,
                     minor_axis=pd.Index(range(w), name='roll'))
    return panel.to_frame().unstack().T.groupby(level=0)

In [352]:
import numpy as np

rolled_df = roll(df, 2)

print(rolled_df.sum())

/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  This is separate from the ipykernel package so we can avoid doing imports until


major       adjClose
symbol              
BORN    12206.038400
CFMS    12691.976000
HEES    76794.969835
HSTM    93742.892804
MR      93685.626268
MTW     83549.816207
MYOS    67928.634800
SYUT    27404.108000
TRIB    51782.822881
YLDE    19344.052547


In [360]:
import statsmodels.api as sm

m.set_index(["symbol", "date"]).rolling(window=60).apply(sm.OLS)
# m.set_index(["symbol", "date"]).rolling(window=60).apply(np.linalg.lstsq)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  This is separate from the ipykernel package so we can avoid doing imports until


AttributeError: 'NoneType' object has no attribute 'shape'

In [348]:
m.set_index(["symbol", "date"]).rolling(3).apply(lambda x: x[0] + x[2], raw=False)

,,adjClose,return,adjClose_spy,return_spy
symbol,date,,,,
MTW,2010-01-05,NaN,NaN,NaN,NaN
HEES,2010-01-05,NaN,NaN,NaN,NaN
TRIB,2010-01-05,14.109117,0.147364,190.679975,0.005294
SYUT,2010-01-05,18.453224,-0.024694,190.679975,0.005294
MR,2010-01-05,38.437107,0.119421,190.679975,0.005294
HSTM,2010-01-05,18.223005,0.022755,190.679975,0.005294
MTW,2010-01-06,44.903582,0.088739,190.747098,0.003351
HEES,2010-01-06,7.839131,0.026364,190.747098,0.003351
TRIB,2010-01-06,14.531318,0.040515,190.814222,0.001408


In [ ]:
df['rs'] = df['returns'].rolling('15d').apply(my_rolling_sharpe)

In [365]:
m = m.set_index(["symbol", "date"])
m

,,adjClose,return,adjClose_spy,return_spy
symbol,date,,,,
MTW,2010-01-05,10.071969,0.073826,95.339988,0.002647
HEES,2010-01-05,4.023224,-0.026777,95.339988,0.002647
TRIB,2010-01-05,4.037149,0.073539,95.339988,0.002647
SYUT,2010-01-05,14.430000,0.002083,95.339988,0.002647
MR,2010-01-05,34.399958,0.045882,95.339988,0.002647
HSTM,2010-01-05,3.793005,0.020672,95.339988,0.002647
MTW,2010-01-06,10.503624,0.042857,95.407111,0.000704
HEES,2010-01-06,4.046126,0.005693,95.407111,0.000704
TRIB,2010-01-06,4.027694,-0.002342,95.407111,0.000704


In [384]:
m["xy"] = list(m[['return_spy', 'return']].itertuples(index=False, name=None))
m

,,adjClose,return,adjClose_spy,return_spy,xy
symbol,date,,,,,
MTW,2010-01-05,10.071969,0.073826,95.339988,0.002647,"(0.0026471366800031853, 0.07382550335792284)"
HEES,2010-01-05,4.023224,-0.026777,95.339988,0.002647,"(0.0026471366800031853, -0.026777469989135837)"
TRIB,2010-01-05,4.037149,0.073539,95.339988,0.002647,"(0.0026471366800031853, 0.07353865493351919)"
SYUT,2010-01-05,14.430000,0.002083,95.339988,0.002647,"(0.0026471366800031853, 0.002083333333333289)"
MR,2010-01-05,34.399958,0.045882,95.339988,0.002647,"(0.0026471366800031853, 0.04588201766746692)"
HSTM,2010-01-05,3.793005,0.020672,95.339988,0.002647,"(0.0026471366800031853, 0.02067183463575302)"
MTW,2010-01-06,10.503624,0.042857,95.407111,0.000704,"(0.0007040394262182491, 0.04285714286182357)"
HEES,2010-01-06,4.046126,0.005693,95.407111,0.000704,"(0.0007040394262182491, 0.005692599627851065)"
TRIB,2010-01-06,4.027694,-0.002342,95.407111,0.000704,"(0.0007040394262182491, -0.0023419203509815255)"


In [390]:
def my_ols(X):
    print(X)
    model_ols = linear_model.LinearRegression()
    model_ols.fit(X[0], X[1])
    return model_ols.coef_[0][0]
    
# m[["return_spy", "return"]].groupby([df.index.get_level_values(0)]).rolling(window=60).apply(my_ols)
# m[["return_spy", "return"]].rolling(window=60).apply(my_ols, raw=False)
m["xy"].rolling(window=60).apply(my_ols)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  if __name__ == '__main__':


TypeError: cannot handle this type -> object

In [367]:
import pandas as pd
from sklearn import linear_model

def rolling_beta(X, y, idx, window=255):

    assert len(X)==len(y)

    out_dates = []
    out_beta = []

    model_ols = linear_model.LinearRegression()

    for iStart in range(0, len(X)-window):        
        iEnd = iStart+window

        model_ols.fit(X[iStart:iEnd], y[iStart:iEnd])

        #store output
        out_dates.append(idx[iEnd])
        out_beta.append(model_ols.coef_[0][0])

    return pd.DataFrame({'beta':out_beta}, index=out_dates)


df_beta = rolling_beta(m['return_spy'].values.reshape(-1, 1), m['return'].values.reshape(-1, 1), m.index.values, 255)

In [368]:
df_beta

,beta
"(SYUT, 2010-03-08 00:00:00)",1.115280
"(MR, 2010-03-08 00:00:00)",1.109084
"(HSTM, 2010-03-08 00:00:00)",1.112196
"(MTW, 2010-03-09 00:00:00)",1.105995
"(HEES, 2010-03-09 00:00:00)",1.106509
"(TRIB, 2010-03-09 00:00:00)",1.104533
"(SYUT, 2010-03-09 00:00:00)",1.100096
"(MR, 2010-03-09 00:00:00)",1.097309
"(HSTM, 2010-03-09 00:00:00)",1.096947
"(MTW, 2010-03-10 00:00:00)",1.097367
